Ghazaleh Mahmoudi

400722156

Report : https://docs.google.com/document/d/1ICvMD6ooM33143_zRGilLj2FND5qhlpowHjX3OTwsFI/edit?usp=sharing

In [1]:
import numpy as np

train = {
        "A":           
            [0, 1, 1, 0,
             1, 0, 0, 1,
             1, 1, 1, 1,
             1, 0, 0, 1,],
            
        "C":
            [0, 1, 1, 1,
             1, 0, 0, 0,
             1, 0, 0, 0,
             0, 1, 1, 1,],
            
        "J":
            [0, 1, 1, 1,
             0, 0, 1, 0,
             1, 0, 1, 0,
             1, 1, 1, 0],
            
        "L":
            [1, 0, 0, 0,
             1, 0, 0, 0,
             1, 0, 0, 0,
             1, 1, 1, 1,]}

test = { 
        "O":
        [0, 1, 1, 0,
         1, 0, 0, 1,
         1, 0, 0, 1,
         0, 1, 1, 0,]}

In [2]:

def ART_algorithm(dataset, ro):
    F2_unit_number = 4
    weights = np.ones((F2_unit_number, 16))
    active_prototype_number = 0
    cluster = []
    map_cluster_alphabet = {}

    # E is each alphabet in dataset 
    for alphabet, E in dataset.items():
        print(f"input alphabet is {alphabet} : {E}")
        test_pass = False
        
        # 1. P0 = E0
        if active_prototype_number == 0:
            weights[active_prototype_number] = E
            active_prototype_number +=1
            cluster.append([alphabet])

            print(f"winner and active prototypes : {[w for w in weights[:active_prototype_number]]}")
            print("weight : ", weights)
            print("cluster : ", cluster)
            print("-------------------------------------------------------")
            map_cluster_alphabet[alphabet]=0
            continue
        
        # 2. check if E is match any prototype
        # 2.1 find closest prototype from active prototypes
        distances = np.linalg.norm(weights[0:active_prototype_number] - np.array(E), axis=1)
        closest_prototype = np.argmin(distances)

        print(f"distance with active protoypes {distances}, closest prototype : p{closest_prototype}")
        
        #product and multiply, |E AND P|/|E| 
        # 3. Vigilance test, if |E AND P|/|E| > ro, then E is match P
        similarity = 1.0*np.dot(E, weights[closest_prototype])/E.count(1)
        print(f"Similarity p{closest_prototype} with {alphabet}: {similarity}")
        
        # 3.1 Vigilance test pass, if |E AND P|/|E| > ro, then E is match P
        if similarity > ro:
            cluster[closest_prototype].append(alphabet)
            print(f"winner prototypes is p{closest_prototype}: {weights[closest_prototype]}")
            map_cluster_alphabet[alphabet] = closest_prototype
            test_pass = True

            # 3.2 update P
            weights[closest_prototype] = np.multiply(weights[closest_prototype], E)
            print("vigilance test pass and winner prototype is updated.")
            
        # 4. Vigilance test fail,  if E is not in any prototype, create new prototypes
        if test_pass == False:
            weights[active_prototype_number] = E
            print(f"winner prototypes is p{active_prototype_number} : {weights[active_prototype_number]}")
            cluster.append([alphabet])
            map_cluster_alphabet[alphabet] = active_prototype_number
            active_prototype_number += 1
            test_pass = False
            print("vigilance test fail and new prototype is added.")

        
        print("active prototypes : ", [w for w in weights[:active_prototype_number]])
        print("weight : ", weights)
        print("cluster : ", cluster)
        print("-------------------------------------------------------------")
    
    print("map cluster alphabet : ", map_cluster_alphabet)
    
    # 4. convergence check
    change = "True"
    last_alphabet = "L"
    while change:

        for alphabet, E in dataset.items():
            distances = np.linalg.norm(weights[:active_prototype_number] - np.array(E), axis=1)
            # distances = [dist(p, self.training_vector[i]) for p in self.prototypes]
            closest_prototype = np.argmin(distances)
            print(f"closest prototype for {alphabet} is p{closest_prototype} and distance is {distances[closest_prototype]} and cluster is {cluster[closest_prototype]}")
            if closest_prototype != map_cluster_alphabet[alphabet]:
                change = True
            elif closest_prototype == map_cluster_alphabet[alphabet] and alphabet == last_alphabet:
                change = False
                
        print("Convergence check : ", not change)
    
    # 6. return winner prototype and cluster
    print("active prototypes : ", [w for w in weights[:active_prototype_number]])
    print("weight : ", weights)
    print("cluster : ", cluster)
    print("--- ----- ---- --- --- --- --- --- --- --- --- --- --- --- --- --- ---")
    
    return weights[:active_prototype_number], cluster

In [19]:
def ART_classify(pattern, prototypes, cluster, ro, alphabet):
    distances = np.linalg.norm(prototypes - np.array(pattern), axis=1)
            # distances = [dist(p, self.training_vector[i]) for p in self.prototypes]
    closest_prototype = np.argmin(distances)
    
    return f"match p{closest_prototype} in cluster {cluster[closest_prototype]} with distance {distances[closest_prototype]}"
        
    # for p_index, p in enumerate(prototypes):
    #     similarity = 1.0*np.dot(pattern, p)/pattern.count(1)
    #     print(f"Similarity {alphabet} with p{p_index}, {cluster[p_index]} : {similarity}")
    #     if similarity > ro:
    #         return f"match p{p_index} in cluster {cluster[p_index]}"
    # return "not match"

In [20]:
prototypes, cluster = ART_algorithm(train, ro=0.1)
print(f'pattern "O" classify result: {ART_classify(test["O"], prototypes, cluster, ro=0.1,  alphabet="O")}')

input alphabet is A : [0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1]
winner and active prototypes : [array([0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.])]
weight :  [[0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
cluster :  [['A']]
-------------------------------------------------------
input alphabet is C : [0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1]
distance with active protoypes [2.82842712], closest prototype : p0
Similarity p0 with C: 0.625
winner prototypes is p0: [0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
vigilance test pass and winner prototype is updated.
active prototypes :  [array([0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.])]
weight :  [[0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

In [21]:
prototypes, cluster = ART_algorithm(train, ro=0.2)
print(f'pattern "O" classify result: {ART_classify(test["O"], prototypes, cluster, ro=0.2,  alphabet="O")}')

input alphabet is A : [0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1]
winner and active prototypes : [array([0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.])]
weight :  [[0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
cluster :  [['A']]
-------------------------------------------------------
input alphabet is C : [0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1]
distance with active protoypes [2.82842712], closest prototype : p0
Similarity p0 with C: 0.625
winner prototypes is p0: [0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
vigilance test pass and winner prototype is updated.
active prototypes :  [array([0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.])]
weight :  [[0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

In [22]:
prototypes, cluster = ART_algorithm(train, ro=0.5)
print(f'pattern "O" classify result: {ART_classify(test["O"], prototypes, cluster, ro=0.5,  alphabet="O")}')

input alphabet is A : [0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1]
winner and active prototypes : [array([0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.])]
weight :  [[0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
cluster :  [['A']]
-------------------------------------------------------
input alphabet is C : [0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1]
distance with active protoypes [2.82842712], closest prototype : p0
Similarity p0 with C: 0.625
winner prototypes is p0: [0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
vigilance test pass and winner prototype is updated.
active prototypes :  [array([0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.])]
weight :  [[0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

In [23]:
prototypes, cluster = ART_algorithm(train, ro=0.7)
print(f'ro : 0.7 -> number of cluster : {len(cluster)}')
print(f'pattern "O" classify result: {ART_classify(test["O"], prototypes, cluster, ro=0.7,  alphabet="O")}')

input alphabet is A : [0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1]
winner and active prototypes : [array([0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.])]
weight :  [[0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
cluster :  [['A']]
-------------------------------------------------------
input alphabet is C : [0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1]
distance with active protoypes [2.82842712], closest prototype : p0
Similarity p0 with C: 0.625
winner prototypes is p1 : [0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1.]
vigilance test fail and new prototype is added.
active prototypes :  [array([0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.]), array([0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1.])]
weight :  [[0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [0. 1. 1. 1. 

In [24]:
prototypes, cluster = ART_algorithm(train, ro=0.9)
print(f'ro : 0.9 -> number of cluster : {len(cluster)}')
print(f'pattern "O" classify result: {ART_classify(test["O"], prototypes, cluster, ro=0.9,  alphabet="O")}')

input alphabet is A : [0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1]
winner and active prototypes : [array([0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.])]
weight :  [[0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
cluster :  [['A']]
-------------------------------------------------------
input alphabet is C : [0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1]
distance with active protoypes [2.82842712], closest prototype : p0
Similarity p0 with C: 0.625
winner prototypes is p1 : [0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1.]
vigilance test fail and new prototype is added.
active prototypes :  [array([0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.]), array([0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1.])]
weight :  [[0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
 [0. 1. 1. 1. 

In [9]:
# for ro in np.arange(0.1, 1, 0.2):
#     prototypes, cluster = ART_algorithm(train, ro)
#     print(f'ro : {ro} -> number of cluster : {len(cluster)}')
#     print("-----------------------------------------------------")